#### Making Fub predictions for the Tox21 library

- Used Physprop collection from genra_dev_v5 to extract OPERA predictions
- Downloaded ToxPrints from the EPA CompTox Chemicals Dashboard
- Used PadelPy to compute PubChem fingerprints


Importing relevant libraries and data files

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
import os
import sys

In [2]:
TOP = os.getcwd().replace('notebooks', '')

In [3]:
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'

In [4]:
models_dir = TOP + 'models/'

In [5]:
tox21 = pd.read_excel(external_dir+'Tox21.xlsx', sheet_name = 'Worksheet1')

In [6]:
tox21.head()

,DTXSID,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,DATA_SOURCES,NUMBER_OF_PUBMED_ARTICLES,PUBCHEM_DATA_SOURCES,CPDAT_COUNT
0,DTXSID7020005,Acetamide,60-35-5,DLFVBJFMPXGRIB-UHFFFAOYSA-N,Acetamide,CC(N)=O,"InChI=1S/C2H5NO/c1-2(3)4/h1H3,(H2,3,4)",C2H5NO,59.068,59.0371,136,275,289,78
1,DTXSID2020006,Acetaminophen,103-90-2,RZVAJINKPMORJF-UHFFFAOYSA-N,N-(4-Hydroxyphenyl)acetamide,CC(=O)NC1=CC=C(O)C=C1,InChI=1S/C8H9NO2/c1-6(10)9-7-2-4-8(11)5-3-7/h2...,C8H9NO2,151.165,151.063,199,15759,396,192
2,DTXSID7020007,Acetohexamide,968-81-0,VGZSUPCWNCWDAN-UHFFFAOYSA-N,4-Acetyl-N-(cyclohexylcarbamoyl)benzene-1-sulf...,CC(=O)C1=CC=C(C=C1)S(=O)(=O)NC(=O)NC1CCCCC1,InChI=1S/C15H20N2O4S/c1-11(18)12-7-9-14(10-8-1...,C15H20N2O4S,324.4,324.114,82,235,153,1
3,DTXSID7020009,Acetonitrile,75-05-8,WEVYAHXRMPXWCK-UHFFFAOYSA-N,Acetonitrile,CC#N,InChI=1S/C2H3N/c1-2-3/h1H3,C2H3N,41.053,41.0265,165,3174,1110,318
4,DTXSID6020014,Dehydroacetic acid,520-45-6,PGRHXDWITVMQBC-UHFFFAOYSA-N,"3-Acetyl-6-methyl-2H-pyran-2,4(3H)-dione",CC(=O)C1C(=O)OC(C)=CC1=O,InChI=1/C8H8O4/c1-4-3-6(10)7(5(2)9)8(11)12-4/h...,C8H8O4,168.148,168.042,112,71,141,28


In [7]:
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import r2_score
import pickle
import glob

In [8]:
def normalizeDescriptors(X):
    scaler = preprocessing.StandardScaler().fit(X)
    transformed = scaler.transform(X)
    x_norm = pd.DataFrame(transformed, index = X.index) 
    x_norm.columns = X.columns
    return(x_norm)


import descriptor files

In [9]:
pubchem = pd.read_csv(interim_dir+'Tox21_pubchem.csv')

In [10]:
pubchem.rename(columns = {'Unnamed: 0' : 'dsstox_sid'}, inplace = True)

In [11]:
pubchem.dsstox_sid.value_counts().sort_values(ascending = False)

DTXSID4025662    1
DTXSID5020738    1
DTXSID1045536    1
DTXSID6021901    1
DTXSID9057684    1
                ..
DTXSID3048782    1
DTXSID9022603    1
DTXSID5045467    1
DTXSID8046799    1
DTXSID4021397    1
Name: dsstox_sid, Length: 8604, dtype: int64

In [12]:
pubchem.set_index('dsstox_sid', inplace = True)

In [13]:
txps = pd.read_csv(interim_dir+'tox21_txps_all.csv')

In [14]:
txps.head()

,Unnamed: 0,INPUT,DTXSID,PREFERRED_NAME,atom:element_main_group,atom:element_metal_group_I_II,atom:element_metal_group_III,atom:element_metal_metalloid,atom:element_metal_poor_metal,atom:element_metal_transistion_metal,...,ring:polycycle_bicyclo_propene,ring:polycycle_spiro_[2.2]pentane,ring:polycycle_spiro_[2.5]octane,ring:polycycle_spiro_[4.5]decane,ring:polycycle_spiro_1_4-dioxaspiro[4.5]decane,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene
0,0,DTXSID7020005,DTXSID7020005,Acetamide,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,DTXSID2020006,DTXSID2020006,Acetaminophen,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,DTXSID7020007,DTXSID7020007,Acetohexamide,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,DTXSID7020009,DTXSID7020009,Acetonitrile,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,DTXSID6020014,DTXSID6020014,Dehydroacetic acid,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
txps.set_index('INPUT', inplace = True)

txps.drop(['Unnamed: 0','DTXSID', 'PREFERRED_NAME'], axis = 1, inplace = True)
txps.head()

,atom:element_main_group,atom:element_metal_group_I_II,atom:element_metal_group_III,atom:element_metal_metalloid,atom:element_metal_poor_metal,atom:element_metal_transistion_metal,atom:element_noble_gas,bond:C#N_cyano_acylcyanide,bond:C#N_cyano_cyanamide,bond:C#N_cyano_cyanohydrin,...,ring:polycycle_bicyclo_propene,ring:polycycle_spiro_[2.2]pentane,ring:polycycle_spiro_[2.5]octane,ring:polycycle_spiro_[4.5]decane,ring:polycycle_spiro_1_4-dioxaspiro[4.5]decane,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene
INPUT,,,,,,,,,,,,,,,,,,,,,
DTXSID7020005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID2020006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID7020007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID7020009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID6020014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_opera = pd.read_csv(interim_dir+'OPERA_TOX21.csv', index_col='dsstox_sid')
df_opera

,Unnamed: 0,OPERA_LogP,OPERA_PKAA,OPERA_PKAB
dsstox_sid,,,,
DTXSID0020022,0,3.700590,NaN,NaN
DTXSID0020024,1,0.752342,NaN,NaN
DTXSID0020105,2,-2.164240,5.42029,NaN
DTXSID0020107,3,-1.204540,7.56472,NaN
DTXSID0020151,4,1.962760,NaN,NaN
...,...,...,...,...
DTXSID9057842,8398,2.523990,4.24546,NaN
DTXSID9057844,8399,7.929440,NaN,3.05035
DTXSID9057846,8400,2.994230,-1.62208,NaN


In [17]:
df_opera.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [18]:
df_opera.columns = ['LogP_pred','pKa_a_pred', 'pKa_b_pred']

In [19]:
df_opera['pKa_pred']=df_opera[['pKa_a_pred','pKa_b_pred']].min(axis=1)


In [20]:
df_opera.head()

,LogP_pred,pKa_a_pred,pKa_b_pred,pKa_pred
dsstox_sid,,,,
DTXSID0020022,3.700590,NaN,NaN,NaN
DTXSID0020024,0.752342,NaN,NaN,NaN
DTXSID0020105,-2.164240,5.42029,NaN,5.42029
DTXSID0020107,-1.204540,7.56472,NaN,7.56472
DTXSID0020151,1.962760,NaN,NaN,NaN


In [21]:
df_opera = df_opera[~df_opera.index.duplicated(keep='first')]

In [22]:
df_opera = df_opera.dropna(subset=['pKa_pred','LogP_pred']) #add1
df_opera.fillna(0, inplace=True) 

In [23]:
opera_scaler = pickle.load(open(models_dir+'opera_scaler_v2.sav', 'rb'))

In [24]:
# Normalize opera properties based on transformation scaler vector from the base models
opera_scaled = opera_scaler.transform(df_opera)
opera = pd.DataFrame(opera_scaled, index = df_opera.index) 
opera.columns = df_opera.columns
opera = opera[['pKa_pred','LogP_pred']]


In [25]:
desc = pd.read_csv(external_dir+'Human.Funbound.plasma_Features_v2.csv')

In [26]:
desc

,Unnamed: 0,Fingerprints,opera,Padel+CDK
0,0,"['bitvector2', 'bitvector12', 'bitvector15', '...","['LogP_pred', 'pKa_a_pred', 'pKa_b_pred', 'pKa...","['nN', 'nO', 'nS', 'nP', 'nF', 'nCl', 'nBr', '..."


In [27]:
ids = list(set(pubchem.index & txps.index))
txps = txps.loc[ids]
pubchem = pubchem.loc[ids]
fingerprints = pd.concat([pubchem,txps ], axis =1)

In [28]:
fingerprints.head()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,ring:polycycle_bicyclo_propene,ring:polycycle_spiro_[2.2]pentane,ring:polycycle_spiro_[2.5]octane,ring:polycycle_spiro_[4.5]decane,ring:polycycle_spiro_1_4-dioxaspiro[4.5]decane,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene
DTXSID4025662,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID8024498,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID6024832,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID5024059,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID9032589,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
fingerprints.columns = fingerprints.columns.str.replace('PubchemFP', 'bitvector')

In [30]:
fingerprints

,bitvector0,bitvector1,bitvector2,bitvector3,bitvector4,bitvector5,bitvector6,bitvector7,bitvector8,bitvector9,...,ring:polycycle_bicyclo_propene,ring:polycycle_spiro_[2.2]pentane,ring:polycycle_spiro_[2.5]octane,ring:polycycle_spiro_[4.5]decane,ring:polycycle_spiro_1_4-dioxaspiro[4.5]decane,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene
DTXSID4025662,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID8024498,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID6024832,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID5024059,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID9032589,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID8021808,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID5045467,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID9021922,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DTXSID9047099,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Select fingerprints from the feature file
retain = [str(val.replace("'", "").replace(" ", "")) for val in desc.loc[0,'Fingerprints'].split(',')]
retain[0] = retain[0].replace("[", "")
retain[len(retain)-1] = retain[len(retain)-1].replace("c]",'c')
fingerprints_fub = fingerprints.loc[:,retain]

In [32]:
fingerprints_fub

,bitvector2,bitvector12,bitvector15,bitvector16,bitvector19,bitvector20,bitvector33,bitvector37,bitvector143,bitvector145,...,bitvector712,bond:CN_amine_aliphatic_generic,bond:CN_amine_ter-N_aliphatic,bond:COH_alcohol_generic,bond:CX_halide_aromatic-X_generic,chain:alkaneCyclic_ethyl_C2_(connect_noZ),chain:alkaneLinear_ethyl_C2(H_gt_1),chain:alkaneLinear_ethyl_C2_(connect_noZ_CN=4),chain:aromaticAlkane_Ph-C1_acyclic_connect_noDblBd,ring:hetero_[6]_N_pyridine_generic
DTXSID4025662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID8024498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
DTXSID6024832,0,0,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
DTXSID5024059,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
DTXSID9032589,1,1,0,0,1,1,0,0,0,0,...,1,0,0,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID8021808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
DTXSID5045467,1,1,1,0,1,1,1,0,1,0,...,1,1,0,1,0,1,0,0,0,0
DTXSID9021922,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
DTXSID9047099,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0


In [33]:
opera_ = opera.loc[ids]

/home/grace/anaconda3/envs/httk/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [34]:
opera_.head()

,pKa_pred,LogP_pred
dsstox_sid,,
DTXSID4025662,NaN,NaN
DTXSID8024498,1.345801,-0.961086
DTXSID6024832,-0.056242,0.210511
DTXSID5024059,-0.737135,-0.077715
DTXSID9032589,-0.506315,2.074383


In [35]:
descriptors = pd.concat([fingerprints_fub, opera_], axis=1).dropna(axis=0, how='any')

In [36]:
descriptors.shape

(6279, 82)

Load sklearn pickle files

In [37]:
fub_rf = pickle.load(open(models_dir+'fub_rf_v2.sav', 'rb'))
fub_svr = pickle.load(open(models_dir+'fub_svr_v2.sav', 'rb'))

In [38]:
len(fub_rf.feature_importances_)

82

In [39]:
predicted_Fub = pd.DataFrame(1/(1+10**fub_rf.predict(descriptors)), descriptors.index )
predicted_Fub.columns = ['pred_Fub_rf']
predicted_Fub_2 = pd.DataFrame(1/(1+10**fub_svr.predict(descriptors)), descriptors.index )
predicted_Fub_2.columns = ['pred_Fub_svr']
predicted_Fub_all = pd.concat([predicted_Fub, predicted_Fub_2], axis = 1)
predicted_Fub_all['Consensus (SVM,RF)'] = predicted_Fub_all[['pred_Fub_svr', 'pred_Fub_rf']].mean(axis = 1)

predicted_Fub_all.head()

,pred_Fub_rf,pred_Fub_svr,"Consensus (SVM,RF)"
dsstox_sid,,,
DTXSID8024498,0.593446,0.593992,0.593719
DTXSID6024832,0.014976,0.016773,0.015875
DTXSID5024059,0.136417,0.136301,0.136359
DTXSID9032589,0.002287,0.001652,0.001969
DTXSID9020112,0.118899,0.081741,0.100320


In [40]:
writer = pd.ExcelWriter(external_dir+'Tox21_httk_Fub_predictions.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.

predicted_Fub_all.to_excel(writer, sheet_name = 'Tox21_Fub_predictions')


writer.save()

#### Creating one file with all the predictions

In [41]:
tox21.head()

,DTXSID,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,DATA_SOURCES,NUMBER_OF_PUBMED_ARTICLES,PUBCHEM_DATA_SOURCES,CPDAT_COUNT
0,DTXSID7020005,Acetamide,60-35-5,DLFVBJFMPXGRIB-UHFFFAOYSA-N,Acetamide,CC(N)=O,"InChI=1S/C2H5NO/c1-2(3)4/h1H3,(H2,3,4)",C2H5NO,59.068,59.0371,136,275,289,78
1,DTXSID2020006,Acetaminophen,103-90-2,RZVAJINKPMORJF-UHFFFAOYSA-N,N-(4-Hydroxyphenyl)acetamide,CC(=O)NC1=CC=C(O)C=C1,InChI=1S/C8H9NO2/c1-6(10)9-7-2-4-8(11)5-3-7/h2...,C8H9NO2,151.165,151.063,199,15759,396,192
2,DTXSID7020007,Acetohexamide,968-81-0,VGZSUPCWNCWDAN-UHFFFAOYSA-N,4-Acetyl-N-(cyclohexylcarbamoyl)benzene-1-sulf...,CC(=O)C1=CC=C(C=C1)S(=O)(=O)NC(=O)NC1CCCCC1,InChI=1S/C15H20N2O4S/c1-11(18)12-7-9-14(10-8-1...,C15H20N2O4S,324.4,324.114,82,235,153,1
3,DTXSID7020009,Acetonitrile,75-05-8,WEVYAHXRMPXWCK-UHFFFAOYSA-N,Acetonitrile,CC#N,InChI=1S/C2H3N/c1-2-3/h1H3,C2H3N,41.053,41.0265,165,3174,1110,318
4,DTXSID6020014,Dehydroacetic acid,520-45-6,PGRHXDWITVMQBC-UHFFFAOYSA-N,"3-Acetyl-6-methyl-2H-pyran-2,4(3H)-dione",CC(=O)C1C(=O)OC(C)=CC1=O,InChI=1/C8H8O4/c1-4-3-6(10)7(5(2)9)8(11)12-4/h...,C8H8O4,168.148,168.042,112,71,141,28


In [47]:
predicted_clint_rf = pd.read_excel(external_dir+'Tox21_httk_CLint_reg_predictions.xlsx', index_col = 0, sheet_name = 'Tox21_Clint_regression_predictions')

In [48]:
predicted_clint_rf.head()

,pred_clint_rf
DTXSID2026943,7.663445
DTXSID5057622,8.432165
DTXSID7045948,5.197866
DTXSID0047408,19.148468
DTXSID1048887,9.326406


In [49]:
predicted_Clint_cls = pd.read_excel(external_dir+'Tox21_httk_Clint_cls_predictions.xlsx', index_col = 0, sheet_name = 'Tox21_Clint_cls_predictions')

In [51]:
predicted_Clint_cls.head()

,Clint Prediction (Bin)
DTXSID0020020,Medium
DTXSID0020070,Low
DTXSID0020072,Medium
DTXSID0020074,Low
DTXSID0020076,Medium


In [53]:
writer = pd.ExcelWriter(external_dir+'Tox21_httk_predictions.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.
tox21.to_excel(writer, sheet_name = 'Tox21_list')
predicted_Fub_all.to_excel(writer, sheet_name = 'Tox21_Fub_predictions')
predicted_clint_rf.to_excel(writer, sheet_name = 'Tox21_Clint_reg_predictions')
predicted_Clint_cls.to_excel(writer, sheet_name = 'Tox21_Clint_cls_predictions')

writer.save()